# Experiments with backtesting.py
This notebook is to explore the `backtesting.py` to evaluate strategy:
1. Based on conventional TA (talib etc.)
2. See how can we introduce ML model as TA

Install ta-lib on Mac with brew first `brew install ta-lib` and then `pip install TA-Lib`

In [1]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import yfinance as yf
import talib 
from utils import get_stock_data
import time
import warnings

/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### 1. Standard Examples
First, use standard example; where goal is just to see how we can use the library. In the following, we will use SMA as an indicator which is already available in `backtesting.py`. We will also use stop loss and take profit to limit the exposure and to ensure risk-reward ratio.

In [2]:
class SmaCrossWithSLTP(Strategy):
    short_window = 10  # Fast SMA
    long_window = 20   # Slow SMA
    stop_loss_pct = 0.02   # 2% Stop Loss
    take_profit_pct = 0.04  # 4% Take Profit
    
    # All the necessary indicator needs to define in the init method
    def init(self):
        self.sma_short = self.I(SMA, self.data.Close, self.short_window)
        self.sma_long = self.I(SMA, self.data.Close, self.long_window)
    
    # It defines the logic of the strategy
    def next(self):
        price = self.data.Close[-1]  # Current price
        if crossover(self.sma_short, self.sma_long):  # Buy condition
            stop_loss = price * (1 - self.stop_loss_pct)
            take_profit = price * (1 + self.take_profit_pct)
            self.buy(sl=stop_loss, tp=take_profit)  # Order with SL and TP
        
        elif crossover(self.sma_long, self.sma_short):  # Sell condition
            self.position.close()  # Close existing position

# Load sample data which comes with the library to test
df = GOOG

# Run Backtest
bt = Backtest(df, SmaCrossWithSLTP, cash=10000, commission=0.002)
stats = bt.run()
print(stats)
# bt.plot()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                     9.82309
Equity Final [$]                  11532.01791
Equity Peak [$]                   12667.87271
Commissions [$]                    2041.36509
Return [%]                           15.32018
Buy & Hold Return [%]               607.37036
Return (Ann.) [%]                     1.68634
Volatility (Ann.) [%]                 6.32304
CAGR [%]                              1.15945
Sharpe Ratio                           0.2667
Sortino Ratio                         0.40031
Calmar Ratio                          0.14875
Alpha [%]                            -4.51391
Beta                                  0.03266
Max. Drawdown [%]                   -11.33648
Avg. Drawdown [%]                    -3.53535
Max. Drawdown Duration      742 days 00:00:00
Avg. Drawdown Duration      172 days 00:00:00
# Trades                          

## 2a. Integrating other indicators from TA-LIB
Now, moving to `ta-lib` for more indicators. In the following, we will use RSI along with SL and TP.

In [3]:
class RsiSmaStrategy(Strategy):
    rsi_period = 14    # RSI period
    rsi_buy = 30       # RSI oversold threshold
    rsi_sell = 70      # RSI overbought threshold
    stop_loss_pct = 0.02  # 2% stop-loss
    take_profit_pct = 0.05  # 5% take-profit

    def init(self):    
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Entry Condition: RSI below 30 & SMA crossover
        if self.rsi[-1] < self.rsi_buy :
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  # Stop-Loss 2% below entry
            tp = price * (1 + self.take_profit_pct)  # Take-Profit 5% above entry
            self.buy(sl=sl, tp=tp)  # Place order with SL & TP

        # Exit Condition: RSI above 70 OR SMA crossover in opposite direction
        elif self.rsi[-1] > self.rsi_sell:
            self.position.close()

df = get_stock_data("MSFT", interval="15m", start="2020-01-01", end="2025-03-20")

# Run Backtest
bt = Backtest(df, RsiSmaStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
# Save HTML file for plots for in-browser visualization
#bt.plot(filename="rsi_strategy.html")

YF.download() has changed argument auto_adjust default to True


Backtest.run:   0%|          | 0/765 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=76: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=77: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=78: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=79: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=80: Bro

Start                     2025-02-25 14:30...
End                       2025-04-07 19:45...
Duration                     41 days 05:15:00
Exposure Time [%]                    63.97436
Equity Final [$]                  10245.71392
Equity Peak [$]                   10362.71824
Commissions [$]                     394.83137
Return [%]                            2.45714
Buy & Hold Return [%]               -10.19317
Return (Ann.) [%]                    22.61813
Volatility (Ann.) [%]                 35.7751
CAGR [%]                             15.99846
Sharpe Ratio                          0.63223
Sortino Ratio                         1.14479
Calmar Ratio                           3.4928
Alpha [%]                             7.79372
Beta                                  0.52354
Max. Drawdown [%]                    -6.47564
Avg. Drawdown [%]                     -1.8749
Max. Drawdown Duration       12 days 19:15:00
Avg. Drawdown Duration        3 days 08:11:00
# Trades                          

## 2b. Using pandas_ta  
`pandas_ta` is a simpler wrapper for dataframe. Some of the TA are build upon above-used `ta-lib` so install both.

In [4]:
class RsiSmaStrategy(Strategy):
    rsi_period = 14
    rsi_buy = 30
    rsi_sell = 70
    stop_loss_pct = 0.02
    take_profit_pct = 0.05

    def init(self):
        # Compute RSI using pandas_ta
        rsi_series = self.data.df.ta.rsi(length=self.rsi_period)
        self.rsi = self.I(lambda: rsi_series)

    def next(self):
        if self.rsi[-1] < self.rsi_buy:
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)
            tp = price * (1 + self.take_profit_pct)
            self.buy(sl=sl, tp=tp)

        elif self.rsi[-1] > self.rsi_sell:
            self.position.close()

# Assuming get_stock_data returns a DataFrame with OHLCV
df = get_stock_data("MSFT", interval="15m", start="2020-01-01", end="2025-03-20")

# Run Backtest
bt = Backtest(df, RsiSmaStrategy, cash=10000, commission=0.002)
stats = bt.run()

print(stats)
# bt.plot(filename="rsi_strategy.html")


Backtest.run:   0%|          | 0/765 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=76: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=77: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=78: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=79: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=80: Bro

Start                     2025-02-25 14:30...
End                       2025-04-07 19:45...
Duration                     41 days 05:15:00
Exposure Time [%]                    63.97436
Equity Final [$]                  10245.71392
Equity Peak [$]                   10362.71824
Commissions [$]                     394.83137
Return [%]                            2.45714
Buy & Hold Return [%]               -10.19317
Return (Ann.) [%]                    22.61813
Volatility (Ann.) [%]                 35.7751
CAGR [%]                             15.99846
Sharpe Ratio                          0.63223
Sortino Ratio                         1.14479
Calmar Ratio                           3.4928
Alpha [%]                             7.79372
Beta                                  0.52354
Max. Drawdown [%]                    -6.47564
Avg. Drawdown [%]                     -1.8749
Max. Drawdown Duration       12 days 19:15:00
Avg. Drawdown Duration        3 days 08:11:00
# Trades                          

#### More TA based strategigies (Bollinger band)

In [17]:
class RsiBollingerStrategy(Strategy):
    rsi_period = 14
    rsi_buy = 30
    rsi_sell = 70
    bb_period = 20
    bb_std = 2
    stop_loss_pct = 0.02
    take_profit_pct = 0.05

    def init(self):
        df = self.data.df

        # Indicators from pandas_ta
        self.rsi = self.I(lambda: df.ta.rsi(length=self.rsi_period))
        bb = df.ta.bbands(length=self.bb_period, std=self.bb_std)

        # Use lower & upper bands from BB
        self.bb_lower = self.I(lambda: bb['BBL_20_2.0'])
        self.bb_upper = self.I(lambda: bb['BBU_20_2.0'])

    def next(self):
        price = self.data.Close[-1]

        # Buy condition: RSI < 30 and price below lower BB
        if self.rsi[-1] < self.rsi_buy and price < self.bb_lower[-1]:
            sl = price * (1 - self.stop_loss_pct)
            tp = price * (1 + self.take_profit_pct)
            self.buy(sl=sl, tp=tp)

        # Sell condition: RSI > 70 or price > upper BB
        elif self.position:
            if self.rsi[-1] > self.rsi_sell or price > self.bb_upper[-1]:
                self.position.close()

# Fetch data
df = get_stock_data("MSFT", interval="15m", start="2020-01-01", end="2025-03-20")

# Run backtest
bt = Backtest(df, RsiBollingerStrategy, cash=10000, commission=0.002)
stats = bt.run()

print(stats)
# bt.plot(filename="rsi_bb_strategy.html")


Backtest.run:   0%|          | 0/760 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=76: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=77: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=78: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=79: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=80: Bro

Start                     2025-02-25 14:30...
End                       2025-04-07 19:45...
Duration                     41 days 05:15:00
Exposure Time [%]                    30.89744
Equity Final [$]                  10149.19723
Equity Peak [$]                   10379.68151
Commissions [$]                     393.43643
Return [%]                            1.49197
Buy & Hold Return [%]               -10.00792
Return (Ann.) [%]                    13.24687
Volatility (Ann.) [%]                26.16851
CAGR [%]                              9.47667
Sharpe Ratio                          0.50621
Sortino Ratio                         0.82574
Calmar Ratio                          2.07612
Alpha [%]                             4.54983
Beta                                  0.30554
Max. Drawdown [%]                    -6.38059
Avg. Drawdown [%]                    -1.01321
Max. Drawdown Duration       31 days 00:30:00
Avg. Drawdown Duration        3 days 12:23:00
# Trades                          

Giving control to LLM for "best" startegy

In [ ]:
class VWAPReversion(Strategy):
    atr_period = 14
    atr_mult = 1.5

    def init(self):
        df = self.data.df

        self.vwap = self.I(lambda: df.ta.vwap(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume']))
        self.atr = self.I(lambda: df.ta.atr(length=self.atr_period))
        self.rsi = self.I(lambda: df.ta.rsi(length=14))

    def next(self):
        price = self.data.Close[-1]
        vwap = self.vwap[-1]
        atr = self.atr[-1]
        rsi = self.rsi[-1]
        hour = self.data.index[-1].hour

        # Only trade during 10:00–14:30 EST
        if hour < 10 or hour > 14:
            return

        dist_from_vwap = abs(price - vwap)

        # LONG: price well below VWAP, signs of reversal
        if (
            price < vwap and
            dist_from_vwap > atr * self.atr_mult and
            self.data.Close[-1] > self.data.Open[-1] and  # green candle
            rsi > 30 and
            not self.position
        ):
            sl = self.data.Low[-1] - atr * 0.5
            tp = vwap
            self.buy(sl=sl, tp=tp)

        # SHORT: price well above VWAP, signs of reversal
        elif (
            price > vwap and
            dist_from_vwap > atr * self.atr_mult and
            self.data.Close[-1] < self.data.Open[-1] and  # red candle
            rsi < 70 and
            not self.position
        ):
            sl = self.data.High[-1] + atr * 0.5
            tp = vwap
            self.sell(sl=sl, tp=tp)

        # Optional early exit if price touches VWAP
        elif self.position:
            if (self.position.is_long and price >= vwap) or (self.position.is_short and price <= vwap):
                self.position.close()


# Fetch MSFT or AAPL
df = get_stock_data("MSFT", interval="15m", start="2022-01-01", end="2025-03-30")

bt = Backtest(df, VWAPReversion, cash=10000, commission=0.001)
stats = bt.run()

print(stats)
# bt.plot("dip_buy_strategy.html")


/var/folders/vd/m8tzs4_92z538swpylf0xtvc0000gn/T/ipykernel_15156/3666103671.py:8: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  self.vwap = self.I(lambda: df.ta.vwap(high=df['High'], low=df['Low'], close=df['Close'], volume=df['Volume']))


Start                     2025-02-25 14:30...
End                       2025-04-07 19:55...
Duration                     41 days 05:25:00
Exposure Time [%]                      2.4359
Equity Final [$]                  10044.33479
Equity Peak [$]                   10104.52497
Commissions [$]                     137.26492
Return [%]                            0.44335
Buy & Hold Return [%]               -10.48641
Return (Ann.) [%]                     3.78579
Volatility (Ann.) [%]                 3.02141
CAGR [%]                              2.74095
Sharpe Ratio                          1.25299
Sortino Ratio                         2.26214
Calmar Ratio                          2.61969
Alpha [%]                              0.2855
Beta                                 -0.01505
Max. Drawdown [%]                    -1.44513
Avg. Drawdown [%]                    -0.26465
Max. Drawdown Duration       12 days 05:15:00
Avg. Drawdown Duration        1 days 11:32:00
# Trades                          

At this point, we have few strategies which works good for some and not so good for others. So let's filter/screen positive stock for us

In [24]:
def run_multi_stock_backtest(tickers, interval="15m", period="30d"):
    results = []

    for ticker in tickers:
        time.sleep(5)
        print(f"Running backtest for {ticker}...")
        try:
            df = get_stock_data(ticker, interval=interval, period=period)
            if df.empty or len(df) < 100:
                print(f"Skipping {ticker} due to insufficient data.")
                continue

            bt = Backtest(df, VWAPReversion, cash=10_000, commission=0.001)
            
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                stats = bt.run()

            results.append({
                "Ticker": ticker,
                "Return [%]": stats['Return [%]'],
                "Win Rate [%]": stats['Win Rate [%]'],
                "Sharpe Ratio": stats.get('Sharpe Ratio', None),
                "Trades": stats['# Trades'],
                "Avg Trade Duration": stats['Avg. Trade Duration'],
                "Exposure Time [%]": stats['Exposure Time [%]'],
            })

        except Exception as e:
            print(f"Error with {ticker}: {e}")

    df_results = pd.DataFrame(results)
    df_results.sort_values(by="Return [%]", ascending=False, inplace=True)
    return df_results

tickers = ["AAPL", "MSFT", "TSLA", "NVDA", "AMZN", "GOOGL", "META", "NFLX", "AMD", "INTC"]
results = run_multi_stock_backtest(tickers, interval="5m", period="10d")
print(results)


Running backtest for AAPL...


Running backtest for MSFT...


Running backtest for TSLA...


Running backtest for NVDA...


Running backtest for AMZN...


...

Running backtest for META...


Running backtest for NFLX...


Running backtest for AMD...


Running backtest for INTC...


  Ticker  Return [%]  Win Rate [%]  Sharpe Ratio  Trades Avg Trade Duration  \
7   NFLX    1.626476     50.000000      2.448598       6    0 days 00:38:00   
5  GOOGL    0.917913     66.666667      4.379874       3    0 days 00:34:00   
9   INTC    0.339031     44.444444      0.465984       9    0 days 08:07:00   
6   META   -0.305072     50.000000     -2.357344       6    0 days 01:09:00   
1   MSFT   -0.423209     60.000000     -3.407536       5    0 days 00:21:00   
0   AAPL   -1.068244     25.000000     -4.773419       4    0 days 00:27:00   
4   AMZN   -1.997109     33.333333     -5.821095      12    0 days 00:25:00   
8    AMD   -4.087307      0.000000    -16.499411       5    0 days 01:17:00   
2   TSLA   -4.106754     14.285714     -4.723235       7    0 days 01:14:00   
3   NVDA   -6.542440     25.000000    -17.743449       4    0 days 17:55:00   

   Exposure Time [%]  
7           6.538462  
5           2.948718  
9          12.564103  
6          11.282051  
1           3.3

## 3. Integrating a black-box indicator (e.g., ML model)
Now we will add a black-box function which can be an ML model which can provide BUY or SELL signal. We will pass the dataframe containing the features and function can use this to predict the position. This then can be used in `next` function.

In [26]:
def dummy_ml_model(df):
    """
    Simulated ML model that predicts BUY signals. We don't consider SELL (short positions).
    :param df: DataFrame of last `N` rows (OHLCV features).
    :return: 1 if BUY signal, else 0.
    """
    # Simulated logic: Buy if the last row closes lower than the first row in the window
    if df.iloc[-1]['Close'] < df.iloc[0]['Close']:  
        return 1  # BUY Signal
    return 0  # No action

class MLTradingStrategy(Strategy):
    feature_window = 5 # window size for features 
    stop_loss_pct = 0.02 
    take_profit_pct = 0.05 

    def init(self):
        pass

    def next(self):
        N = self.feature_window  
        # Ensure enough data to generate features
        if len(self.data.Close) < N:
            return 

        # Extract last `N` rows into a DataFrame
        df = pd.DataFrame({
            "Open": self.data.Open[-N:],
            "High": self.data.High[-N:],
            "Low": self.data.Low[-N:],
            "Close": self.data.Close[-N:],
            "Volume": self.data.Volume[-N:],
        })

        # Pass last `N` rows to ML model
        signal = dummy_ml_model(df)  

        # ML suggests BUY signal
        if signal == 1:  
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  
            tp = price * (1 + self.take_profit_pct) 
            self.buy(sl=sl, tp=tp)  

df = get_stock_data("MSFT", period="1y") 

# Run Backtest
bt = Backtest(df, MLTradingStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
#bt.plot(filename="ml__strategy.html")

Backtest.run:   0%|          | 0/250 [00:00<?, ?it/s]/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=7: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=11: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=12: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=18: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
/Users/ikespand/anaconda3/envs/py311/lib/python3.11/site-packages/backtesting/backtesting.py:955: UserWarning: time=19: Brok

Start                     2024-04-08 00:00:00
End                       2025-04-07 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                    75.69721
Equity Final [$]                   6260.21381
Equity Peak [$]                       10000.0
Commissions [$]                     1382.1176
Return [%]                          -37.39786
Buy & Hold Return [%]               -15.07286
Return (Ann.) [%]                   -37.51457
Volatility (Ann.) [%]                12.37933
CAGR [%]                            -27.69354
Sharpe Ratio                         -3.03042
Sortino Ratio                        -2.25133
Calmar Ratio                         -1.00312
Alpha [%]                           -27.60963
Beta                                  0.64939
Max. Drawdown [%]                   -37.39786
Avg. Drawdown [%]                   -37.39786
Max. Drawdown Duration      360 days 00:00:00
Avg. Drawdown Duration      360 days 00:00:00
# Trades                          

## 4. Integrating ESN model from `1_esn_le2e.ipynb`
Here, our function will look for some threshold above which we can have BUY.

In [27]:
from utils import StockPricePredictor

predictor = StockPricePredictor(model_path="stock_esn_model_7.pkl", 
                                scaler_path="scaler_esn_model_7.pkl")

def if_buy_signal_with_esn(df, threshold=0.02):
    """
    Determines if a BUY signal should be generated based on the ESN model's prediction.
    
    Args:
        df (DataFrame): The dataframe containing the latest stock data.
        threshold (float): The minimum percentage increase required to trigger a BUY signal.
    
    Returns:
        int: 1 if BUY signal is triggered, otherwise 0.
    """
    predicted_close = predictor.predict(df)
    last_close = df.iloc[-1]['Close']
    
    # Check if predicted close is at least (1 + threshold)% higher than last close
    if predicted_close >= last_close * (1 + threshold):
        return 1  # BUY Signal
    
    return 0  # No action


class MLTradingStrategy(Strategy):
    feature_window = 7 # window size for features 
    stop_loss_pct = 0.025 
    take_profit_pct = 0.05 

    def init(self):
        pass

    def next(self):
        N = self.feature_window  
        # Ensure enough data to generate features
        if len(self.data.Close) < N:
            return 

        # Extract last `N` rows into a DataFrame
        df = pd.DataFrame({
            "Open": self.data.Open[-N:],
            "High": self.data.High[-N:],
            "Low": self.data.Low[-N:],
            "Close": self.data.Close[-N:],
            #"Volume": self.data.Volume[-N:],
        })

        # Pass last `N` rows to ML model
        signal = if_buy_signal_with_esn(df)  

        # ML suggests BUY signal
        if signal == 1:  
            price = self.data.Close[-1]
            sl = price * (1 - self.stop_loss_pct)  
            tp = price * (1 + self.take_profit_pct) 
            self.buy(sl=sl, tp=tp)  

df = get_stock_data("MSFT", period="1y") 

# Run Backtest
bt = Backtest(df, MLTradingStrategy, cash=10000, commission=0.002)
stats = bt.run()

# Print Results
print(stats)
# bt.plot(filename="esn_strategy.html")

Using Numpy backend.
                                                                

Start                     2024-04-08 00:00:00
End                       2025-04-07 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                     6.77291
Equity Final [$]                   9004.72607
Equity Peak [$]                       10000.0
Commissions [$]                     112.43713
Return [%]                           -9.95274
Buy & Hold Return [%]               -15.07286
Return (Ann.) [%]                    -9.99034
Volatility (Ann.) [%]                 5.96468
CAGR [%]                             -7.00072
Sharpe Ratio                         -1.67492
Sortino Ratio                        -1.59482
Calmar Ratio                         -1.00378
Alpha [%]                            -9.05735
Beta                                   0.0594
Max. Drawdown [%]                    -9.95274
Avg. Drawdown [%]                    -9.95274
Max. Drawdown Duration      356 days 00:00:00
Avg. Drawdown Duration      356 days 00:00:00
# Trades                          

# Predicting for today

In [28]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')
df = get_stock_data(ticker="MSFT", interval="1d", start="2025-03-20", end=today)
df = df.drop(columns=['Volume'])
df.head(10)

,Open,High,Low,Close
Date,,,,
2025-03-20,385.739990,391.790009,383.279999,386.839996
2025-03-21,383.220001,391.739990,382.799988,391.260010
2025-03-24,395.399994,395.399994,389.809998,393.079987
2025-03-25,393.920013,396.359985,392.640015,395.160004
2025-03-26,395.000000,395.309998,388.570007,389.970001
2025-03-27,390.130005,392.239990,387.399994,390.579987
2025-03-28,388.079987,389.130005,376.929993,378.799988
2025-03-31,372.540009,377.070007,367.239990,375.390015
2025-04-01,374.649994,382.850006,373.230011,382.190002


In [29]:
signal = if_buy_signal_with_esn(df.tail(7)) 
print("signal", signal)
predicted_close = predictor.predict(df.tail(7))
print("predicted_close", predicted_close) #actual close on 31.03.2025 was 

signal 1
predicted_close 374.4777508505535


END OF NOTEBOOK